# Capstone project Data Science

This project notebook is setup for the capstone project of thethe Data Science course.

In [ ]:
print('Hello Capstone Project Course!')

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
import seaborn as sns
%matplotlib inline

In [ ]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_dbc21f765edb4af4abc41f7c6fee8f2d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9a3_kaHLtHd9jk7VEUcLg3Ux-bxWR3dXKU8vHv_Vkq7M',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_dbc21f765edb4af4abc41f7c6fee8f2d.get_object(Bucket='datasciencecapstone-donotdelete-pr-g2ahuzkxhirpn2',Key='Data-Collisions.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_raw= pd.read_csv(body)
df_raw['SEVERITYCODE'].size

### How many attributes

In [ ]:
df_raw.columns.size

### Empty items in 'ROADCOND'

In [ ]:
df_raw.columns

In [ ]:
df_raw[df_raw['ROADCOND'].isna()==True].shape

# Data pre-processing

In [ ]:
df_raw.columns
cols=['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND','WEATHER','COLLISIONTYPE']

### Label encoding
https://pbpython.com/categorical-encoding.html

In [ ]:
cleanup={'ROADCOND': {'Dry':0, 'Wet':1, 'Unknown':2,'Ice':3, 'Snow/Slush':4, 'Other':5, 'Standing Water':6, 'Sand/Mud/Dirt':7,'Oil':8}, 'LIGHTCOND':{'Daylight':0,'Dark - Street Lights On':1, 'Unknown': 2, 'Dusk':3, 'Dawn':4, 'Dark - No Street Lights':5, 'Dark - Street Lights Off':6, 'Other':7, 'Dark - Unknown Lighting': 8},'WEATHER':{'Clear':0,'Raining':1,'Overcast':2,'Unknown':3,'Snowing':4,'Other':5,'Fog/Smog/Smoke':6,'Sleet/Hail/Freezing Rain':7,'Blowing Sand/Dirt':8,'Severe Crosswind':9,'Partly Cloudy':10},'COLLISIONTYPE':{'Parked Car':0,'Angles':1,'Rear Ended':2,'Other':3,'Sideswipe':4,'Left Turn':5, 'Pedestrian':6,'Cycles':7,'Right Turn':8,'Head On':9}
}

In [ ]:
df=df_raw[cols]
df.replace(cleanup, inplace=True)
df.head(3)

In [ ]:
df_filt=df.dropna()
df_filt['SEVERITYCODE'].size

### Label balancing

In [ ]:
df_maj=df_filt[df_filt['SEVERITYCODE']==1]
df_min=df_filt[df_filt['SEVERITYCODE']==2]
print([df_maj.SEVERITYCODE.size,df_min.SEVERITYCODE.size])

In [ ]:
from sklearn.utils import resample
df_maj_downsampled=resample(df_maj,replace=False,n_samples=df_min.SEVERITYCODE.size, random_state=123)

In [ ]:
df_ds=pd.concat([df_maj_downsampled,df_min])
df_ds.SEVERITYCODE.value_counts()

# Initialisation

In [ ]:
X=df_ds[['WEATHER','ROADCOND','LIGHTCOND','WEATHER','COLLISIONTYPE']]
y=df_ds['SEVERITYCODE']

### Normalize

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:3]

### Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score, f1_score, accuracy_score, classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# Train models

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Nearest neighbour there is a pre-step to figure out how many clusters. As I fully want the test set as confirmation I split the training set up into test/train once more just for KNN. ONLY USED FOR CLUSTER NO. ESTIMATION!

In [ ]:
X_train_KNN, X_test_KNN, y_train_KNN, y_test_KNN = train_test_split( X_train, y_train, test_size=0.2, random_state=4)

In [ ]:
"""
Ks = 30
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    # Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train_KNN,y_train_KNN)
    yhat=neigh.predict(X_test_KNN)
    mean_acc[n-1] = accuracy_score(y_test_KNN, yhat)
    std_acc[n-1]=np.std(yhat==y_test_KNN)/np.sqrt(yhat.shape[0])
    
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- std'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()
"""

Found that clusters=15 best!

In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 15).fit(X_train, y_train)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
pTree = DecisionTreeClassifier(criterion="entropy", max_depth = 7)
pTree.fit(X_train,y_train)

### Support vector machine

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
"""
X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split( X_train, y_train, test_size=0.2, random_state=4)

l=[1,3,6,9]
mean_acc = np.zeros((len(l)-1))
std_acc = np.zeros((len(l)-1))
ConfustionMx = [];
for x in l:
    print(x)
    LR = LogisticRegression(C = x, solver='liblinear').fit(X_train_LR,y_train_LR)
    yhat = LR.predict(X_test_LR)
    yhat_prob = LR.predict_proba(X_test_LR)
    print("Logistic Regression evaluation F1-score: ", f1_score(y_test_LR, yhat, average='weighted') )
    mean_acc[n-1] = accuracy_score(y_test_LR, yhat)
    std_acc[n-1]=np.std(yhat==y_test_LR)/np.sqrt(yhat.shape[0])

plt.plot(range(1,len(l)),mean_acc,'g')
plt.fill_between(range(1,len(l)),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- std'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()
"""

The regularization parameter C = 6 appears to fit best.

In [ ]:
LR = LogisticRegression(C = 6, solver='liblinear').fit(X_train,y_train)
LR

# Model evaluation

### KNN

In [ ]:
yhat=neigh.predict(X_test)
print("KNN's evaluation F1-score: ", f1_score(y_test, yhat, average='weighted') )
print("KNN's evaluation Jaccard-score: ",jaccard_similarity_score(y_test, yhat) )

### Decision tree

In [ ]:
yhat = pTree.predict(X_test)
print("DecisionTrees's F1-score: ", f1_score(y_test, yhat, average='weighted')  )
print("DecisionTrees's  evaluation Jaccard-score: ", jaccard_similarity_score(y_test, yhat) )

### Logistic regression

In [ ]:
from sklearn.metrics import log_loss
yhat = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)
print("Logistic Regression evaluation F1-score: ", f1_score(y_test, yhat, average='weighted') )
print("Logistic Regression evaluation Jaccard-score: ", jaccard_similarity_score(y_test, yhat) )
print("Logistic Regression evaluation Logloss-score: ", log_loss(y_test, yhat_prob) )

### SVM

In [ ]:
yhat = clf.predict(X_test)
print("SVM evaluation F1-score: ", f1_score(y_test, yhat, average='weighted')  )
print("SVM evaluation Jaccard-score: ", jaccard_similarity_score(y_test, yhat) )

The end